In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler



In [13]:
data = pd.read_csv('./data/merged_data.csv')

In [14]:
label_encoders = {}
for column in ['Book-Title', 'Book-Author', 'Publisher']:
    label_encoders[column] = LabelEncoder()
    data[column] = label_encoders[column].fit_transform(data[column])

In [15]:
# División de datos en entrenamiento y prueba
train, test = train_test_split(data, test_size=0.2)

In [17]:
# Normalización de datos numéricos
scaler = StandardScaler()
train[['Year-Of-Publication']] = scaler.fit_transform(train[['Year-Of-Publication']])
test[['Year-Of-Publication']] = scaler.transform(test[['Year-Of-Publication']])

In [18]:
import tensorflow as tf

In [22]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(4,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [23]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [24]:
model.fit(train[['Book-Title', 'Book-Author', 'Publisher', 'Year-Of-Publication']], train['Book-Rating'], epochs=5, batch_size=32)

Epoch 1/5
25779/25779 [==============================] - 18s 689us/step - loss: 22138.7910
Epoch 2/5
25779/25779 [==============================] - 18s 710us/step - loss: 1453.7455
Epoch 3/5
25779/25779 [==============================] - 17s 666us/step - loss: 360.1299
Epoch 4/5
25779/25779 [==============================] - 18s 688us/step - loss: 97.4373
Epoch 5/5
25779/25779 [==============================] - 17s 675us/step - loss: 30.5576


In [25]:
user_id = 276725

# Filtrar los libros que el usuario no ha valorado
unrated_books = data[~data['User-ID'].isin([user_id])][['Book-Title', 'Book-Author', 'Publisher', 'Year-Of-Publication']].drop_duplicates()

# Realizar predicciones para los libros no valorados por el usuario
predictions = model.predict(unrated_books)

8401/8401 [==============================] - 5s 541us/step


In [26]:
# Agregar las predicciones al DataFrame de los libros no valorados
unrated_books['Predicted_Rating'] = predictions

# Mostrar los 10 libros con las calificaciones predichas más altas
top_recommendations = unrated_books.sort_values(by='Predicted_Rating', ascending=False).head(10)
print(top_recommendations)


        Book-Title  Book-Author  Publisher  Year-Of-Publication  \
899768        4842          212      16139                 1999   
35404        10512        98618      14181                    0   
680873       10421        99877      13677                    0   
786552       38411       100931      15331                    0   
821169       10836        87263      15864                    0   
576261       21876        96148      15647                    0   
784262       15857        92722      14950                    0   
428397        3731          507      13190                 2000   
528329       33883        99306      15189                    0   
424826       29278       101451      14693                    0   

        Predicted_Rating  
899768          6.931890  
35404           5.422204  
680873          5.413890  
786552          5.345872  
821169          5.294159  
576261          5.268323  
784262          5.252815  
428397          5.249682  
528329          5.1